# Lesson 6 - Task 2 – Sentiment Analysis using NLP

Sentiment analysis is one of the popular downstream applications of Natural Language Processing, which determines the sentiment expressed in a piece of text. The sentiment expressed in a text is usually classified as positive or negative or neutral. You are expected to train a machine learning model to predict the sentiment of Tweets posted about US Airlines.  

## Step 1 - Load the Dataset

In [40]:
import pandas as pd
import nltk

df = pd.read_csv('Tweets.csv')
df.head(5)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [41]:
df.shape

(14640, 15)

Drop unnecessary columns

In [42]:
df = df[["airline_sentiment", "text"]]
df.head(5)

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [43]:
df.groupby('airline_sentiment').describe()

text         \
                  count unique   
airline_sentiment                
negative           9178   9087   
neutral            3099   3067   
positive           2363   2298   

                                                                           
                                                                 top freq  
airline_sentiment                                                          
negative           @AmericanAir that's 16+ extra hours of travel ...    2  
neutral                                           @SouthwestAir sent    5  
positive                                            @JetBlue thanks!    5

## Step 2 - Preprocess Text
(included in the assignment)

In [44]:
import string
import re

from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')
from nltk.corpus import stopwords

nltk.download('punkt')
ps = PorterStemmer()


def clean_text(text):
  text = text.lower()
  text = re.sub(r'http.?://[^\s]+[\s]?', '', text)
  text = nltk.word_tokenize(text)
  y = []

  for i in text:
    if i not in stopwords.words('english'):
      y.append(i)

  text = y[:]
  y.clear()

  for i in text:
    y.append(ps.stem(i))

  return" ".join(y)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Apply the above preprocessing to the 'text' column, and pass the output to 'text_cleaned'

In [45]:
df['text_cleaned'] = df['text'].apply(clean_text)
df.head(5)

,airline_sentiment,text,text_cleaned
0,neutral,@VirginAmerica What @dhepburn said.,@ virginamerica @ dhepburn said .
1,positive,@VirginAmerica plus you've added commercials t...,@ virginamerica plu 've ad commerci experi ......
2,neutral,@VirginAmerica I didn't today... Must mean I n...,@ virginamerica n't today ... must mean need t...
3,negative,@VirginAmerica it's really aggressive to blast...,@ virginamerica 's realli aggress blast obnoxi...
4,negative,@VirginAmerica and it's a really big bad thing...,@ virginamerica 's realli big bad thing


## Step 3 - Feature Extraction

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_vec = TfidfVectorizer(max_features=3000)
x = tf_vec.fit_transform(df['text_cleaned']).toarray()

y = df['airline_sentiment'].values

In [47]:
x.shape

(14640, 3000)

## Step 4 - Train Model

In [48]:
from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size=0.2, random_state=2)

Multinomial Naive Bayes Classifier

In [49]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(xTrain, yTrain)

from sklearn.metrics import accuracy_score
yPred = model.predict(xTest)
print(accuracy_score(yTest, yPred))

0.7219945355191257


Random Forest Classifier

In [50]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(xTrain,yTrain)

yPred = model.predict(xTest)
print(accuracy_score(yTest,yPred))

0.7561475409836066
